<h3>LSTM 베끼기<h3>

먼저 우리에게 주어진 것과 우리가 예측해야 할 것에 대해 토론합시다.        
데이터 세트 정보:        

교육 데이터는 다음과 같습니다.     
1. 날짜 - 모든 품목 판매 날짜     
2. date_block_num - 매월 지정된 번호       
3. shop_id - 모든 shop의 고유 번호      
4. item_id - 모든 항목의 고유 번호     
5. 품목_가격 - 모든 품목의 가격     
6. item_cnt_day - 특정 날짜에 판매된 품목 수       

테스트 데이터는 다음과 같습니다.       
1. ID - 모든 (shop_id, item_id) 쌍에 대해 고유합니다.   
2. shop_id - 모든 shop의 고유 번호     
3. item_id - 모든 항목의 고유 번호    

이제 우리가 예측해야 할 것은 무엇일까요?       
우리는 한 달 동안 각 가게의 한 종류의 품목이 얼마나 많이 팔릴지 예측해야 한다.      
제출물에는 ID와 item_cnt_month 열이 있어야 합니다.        

우리의 접근 방식은 무엇인가?     
우리의 접근은 간단할 것이다.     
우리의 특징은 지난달 데이터를 제외한 매장에서 한 달에 판매된 품목의 수가 될 것이고,       
이는 우리의 모델이 다음 순서를 예측하는 것을 배우는 데 도움이 될 것이기 때문이다.        
그리고 테스트에는 이 같은 데이터 차원이 그대로 유지되는 첫 달을 제외한 월별 판매 품목의 수가 사용됩니다.       
우리의 모델은 다음 순서를 예측할 것이고 우리는 우리의 결과가 될 것이다.         
이것은 꽤 간단한 접근법이지만 시작하기에 좋다. 다른 접근 방식도 시도해 보십시오.         

First of all as we know import required libraries      
먼저 라이브러리를 import 합니다.

In [1]:
import numpy as np
import pandas as pd 
import os

In [2]:
#loading data 

data = "./data"
sales_data = pd.read_csv(data+'/sales_train.csv')
item_cat = pd.read_csv(data+'/item_categories_en.csv')
items = pd.read_csv(data+'/items_en.csv')
shops = pd.read_csv(data+'/shops_en.csv')
sample_submission = pd.read_csv(data+'/sample_submission.csv')
test_data = pd.read_csv(data+'/test.csv')


In [3]:
def basic_eda(df):
    print("----------TOP 5 RECORDS--------")
    print(df.head(5))
    print("----------INFO-----------------")
    print(df.info())
    print("----------Describe-------------")
    print(df.describe())
    print("----------Columns--------------")
    print(df.columns)
    print("----------Data Types-----------")
    print(df.dtypes)
    print("-------Missing Values----------")
    print(df.isnull().sum())
    print("-------NULL values-------------")
    print(df.isna().sum())
    print("-----Shape Of Data-------------")
    print(df.shape)
    
    

In [4]:
# 데이터를 탐색해봅시다.

print("=============================Sales Data=============================")
basic_eda(sales_data)
print("=============================Test data=============================")
basic_eda(test_data)
print("=============================Item Categories=============================")
basic_eda(item_cat)
print("=============================Items=============================")
basic_eda(items)
print("=============================Shops=============================")
basic_eda(shops)
print("=============================Sample Submission=============================")
basic_eda(sample_submission)



=============================Sales Data=============================
----------TOP 5 RECORDS--------
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
----------INFO-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB
None
----------

In [5]:
# date컬럼을 datetime 형태의 컬럼으로 변환합니다.
sales_data['date'] = pd.to_datetime(sales_data['date'],format = '%d.%m.%Y')
display(sales_data)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [6]:
# 이제 데이터를 원하는 형태로 가져올 수 있도록 피벗 테이블을 생성하겠습니다.
# 한 상점에 대한 한 달 동안의 총 항목 수 값을 얻고자 합니다.
# 그래서 우리는 shop_id와 item_id 우리의 인덱스와 date_block_num을 우리의 칼럼으로 만들었다.
# 우리가 원하는 값은 item_cnt_day이며 합계 함수로 사용합니다.

# 비어있는 곳에 0을 채워넣으면서 원핫인코딩으로 바꿔버린다.
dataset = sales_data.pivot_table(index = ['shop_id','item_id'], values = ['item_cnt_day'], columns = ['date_block_num'], fill_value = 0, aggfunc='sum')
dataset

item_cnt_day                              ...                 \
date_block_num            0   1  2  3  4  5  6  7  8  9   ... 24 25 26 27 28   
shop_id item_id                                           ...                  
0       30                 0  31  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        31                 0  11  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        32                 6  10  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        33                 3   3  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        35                 1  14  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
...                      ...  .. .. .. .. .. .. .. .. ..  ... .. .. .. .. ..   
59      22154              1   0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        22155              0   0  0  0  0  0  1  0  0  0  ...  0  0  0  0  0   
        22162              0   0  0  0  0  0  0  0  0  0  ...  0  9  4  1  1   
        22164              0   0  0  0  0  0  0  0  0  0  ...  0  2  1  2  0   
        22167              0   0  0  0  0  0  0  0  0  1  ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
0       30       0  0  0  0  0  
        31       0  0  0  0  0  
        32       0  0  0  0  0  
        33       0  0  0  0  0  
        35       0  0  0  0  0  
...             .. .. .. .. ..  
59      22154    0  0  0  0  0  
        22155    0  0  0  0  0  
        22162    0  0  1  0  0  
        22164    0  1  0  0  0  
        22167    0  0  0  0  0  

[424124 rows x 34 columns]

In [7]:
# 데이터를 쉽게 조작할 수 있도록 인덱스를 재설정합니다.
dataset.reset_index(inplace = True)

# 피벗테이블을 check
dataset

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [8]:
# 피벗테이블 데이터와 test데이터를 합쳐봅시다.
dataset = pd.merge(test_data,dataset,on = ['item_id','shop_id'],how = 'left')
dataset

/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# NaN을 0으로 채웁시다
dataset.fillna(0,inplace = True)
# dataset 체크
dataset

,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# shop_id, item_id 제거 
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 타겟데이터를 제외한 모든 것을 훈련데이터
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)

# 타겟데이터
y_train = dataset.values[:,-1:]

# 타겟데이터를 제외한 모든 것을 테스트데이터
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# 테스트데이터의 타겟데이터
print(X_train.shape,y_train.shape,X_test.shape)


(214200, 33, 1) (214200, 1) (214200, 33, 1)


In [13]:
# LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [14]:
# 모델설계
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error']) # 평균제곱오차 / adam / metrics
my_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
my_model.fit(X_train,y_train,batch_size = 2048,epochs = 20)

Epoch 1/20
105/105 [==============================] - 1s 8ms/step - loss: 30.4577 - mean_squared_error: 30.4577
Epoch 2/20
105/105 [==============================] - 1s 8ms/step - loss: 30.0074 - mean_squared_error: 30.0074
Epoch 3/20
105/105 [==============================] - 1s 7ms/step - loss: 29.7752 - mean_squared_error: 29.7752
Epoch 4/20
105/105 [==============================] - 1s 7ms/step - loss: 29.6412 - mean_squared_error: 29.6412
Epoch 5/20
105/105 [==============================] - 1s 7ms/step - loss: 29.6109 - mean_squared_error: 29.6109
Epoch 6/20
105/105 [==============================] - 1s 8ms/step - loss: 29.5152 - mean_squared_error: 29.5152
Epoch 7/20
105/105 [==============================] - 1s 8ms/step - loss: 29.5155 - mean_squared_error: 29.5155
Epoch 8/20
105/105 [==============================] - 1s 7ms/step - loss: 29.4658 - mean_squared_error: 29.4658
Epoch 9/20
105/105 [==============================] - 1s 7ms/step - loss: 29.3196 - mean_squared_error: 

In [16]:
# 예측값
submission_pfs = my_model.predict(X_test)
submission_pfs

array([[0.5713114 ],
       [0.14590979],
       [0.8842118 ],
       ...,
       [0.10290506],
       [0.14590979],
       [0.07221949]], dtype=float32)

In [17]:
# 0이하 20 이상인 값들은 0이나 20으로 치환
submission_pfs = submission_pfs.clip(0,20) 
submission_pfs

array([[0.5713114 ],
       [0.14590979],
       [0.8842118 ],
       ...,
       [0.10290506],
       [0.14590979],
       [0.07221949]], dtype=float32)

In [18]:
# 제출파일에 맞게 데이터프레임 형성
submission = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_pfs.ravel()})
submission

,ID,item_cnt_month
0,0,0.571311
1,1,0.145910
2,2,0.884212
3,3,0.169459
4,4,0.145910
...,...,...
214195,214195,0.420112
214196,214196,0.145910
214197,214197,0.102905
214198,214198,0.145910


In [19]:
# 데이터프레임 저장
submission.to_csv('sub_pfs2.csv',index = False)